In [ ]:
# PySpark Window functions

# PySpark Window functions are used to calculate results such as the rank, row number e.t.c over a range of input rows

In [ ]:
'''
# Window Functions
    - ranking functions
    - analytic functions
    - aggregate functions

In [2]:
# Testing pyspark Intallation
import findspark
findspark.init('C:\Spark')
findspark.find()

import pyspark
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

spark

In [6]:
# Read CSV File

df = spark.read.option("header",True).csv("data/6.csv")

df.show(5)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|   "Sales"|  3000|
|      Michael|   "Sales"|  4600|
|       Robert|   "Sales"|  4100|
|        Maria| "Finance"|  3000|
|        James|   "Sales"|  3000|
+-------------+----------+------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: string (nullable = true)



In [8]:
# row_number 

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

windowSpec  = Window.partitionBy("department").orderBy("salary")
df.withColumn("row_number",row_number().over(windowSpec)).show(truncate=False)

+-------------+------------+------+----------+
|employee_name|department  |salary|row_number|
+-------------+------------+------+----------+
|Kumar        | "Marketing"| 2000 |1         |
|Jeff         | "Marketing"| 3000 |2         |
|James        | "Sales"    | 3000 |1         |
|James        | "Sales"    | 3000 |2         |
|Robert       | "Sales"    | 4100 |3         |
|Saif         | "Sales"    | 4100 |4         |
|Michael      | "Sales"    | 4600 |5         |
|Maria        | "Finance"  | 3000 |1         |
|Scott        | "Finance"  | 3300 |2         |
|Jen          | "Finance"  | 3900 |3         |
+-------------+------------+------+----------+



In [9]:
# rank Window Function

# rank() window function is used to provide a rank to the result within a window partition.

from pyspark.sql.functions import rank

df.withColumn("rank",rank().over(windowSpec)).show()

+-------------+------------+------+----+
|employee_name|  department|salary|rank|
+-------------+------------+------+----+
|        Kumar| "Marketing"|  2000|   1|
|         Jeff| "Marketing"|  3000|   2|
|        James|     "Sales"|  3000|   1|
|        James|     "Sales"|  3000|   1|
|       Robert|     "Sales"|  4100|   3|
|         Saif|     "Sales"|  4100|   3|
|      Michael|     "Sales"|  4600|   5|
|        Maria|   "Finance"|  3000|   1|
|        Scott|   "Finance"|  3300|   2|
|          Jen|   "Finance"|  3900|   3|
+-------------+------------+------+----+



In [10]:
'''
# dense_rank :
    - dense_rank() window function is used to get the result with rank of rows within a window partition without any gaps.
'''

from pyspark.sql.functions import dense_rank

df.withColumn("dense_rank",dense_rank().over(windowSpec)).show()

+-------------+------------+------+----------+
|employee_name|  department|salary|dense_rank|
+-------------+------------+------+----------+
|        Kumar| "Marketing"|  2000|         1|
|         Jeff| "Marketing"|  3000|         2|
|        James|     "Sales"|  3000|         1|
|        James|     "Sales"|  3000|         1|
|       Robert|     "Sales"|  4100|         2|
|         Saif|     "Sales"|  4100|         2|
|      Michael|     "Sales"|  4600|         3|
|        Maria|   "Finance"|  3000|         1|
|        Scott|   "Finance"|  3300|         2|
|          Jen|   "Finance"|  3900|         3|
+-------------+------------+------+----------+



In [11]:
'''
percent_rank 

'''

from pyspark.sql.functions import percent_rank
    
df.withColumn("percent_rank",percent_rank().over(windowSpec)).show()

+-------------+------------+------+------------+
|employee_name|  department|salary|percent_rank|
+-------------+------------+------+------------+
|        Kumar| "Marketing"|  2000|         0.0|
|         Jeff| "Marketing"|  3000|         1.0|
|        James|     "Sales"|  3000|         0.0|
|        James|     "Sales"|  3000|         0.0|
|       Robert|     "Sales"|  4100|         0.5|
|         Saif|     "Sales"|  4100|         0.5|
|      Michael|     "Sales"|  4600|         1.0|
|        Maria|   "Finance"|  3000|         0.0|
|        Scott|   "Finance"|  3300|         0.5|
|          Jen|   "Finance"|  3900|         1.0|
+-------------+------------+------+------------+



In [12]:
# ntile Window Function
# ntile() window function returns the relative rank of result rows within a window partition

from pyspark.sql.functions import ntile
df.withColumn("ntile",ntile(2).over(windowSpec)).show()

+-------------+------------+------+-----+
|employee_name|  department|salary|ntile|
+-------------+------------+------+-----+
|        Kumar| "Marketing"|  2000|    1|
|         Jeff| "Marketing"|  3000|    2|
|        James|     "Sales"|  3000|    1|
|        James|     "Sales"|  3000|    1|
|       Robert|     "Sales"|  4100|    1|
|         Saif|     "Sales"|  4100|    2|
|      Michael|     "Sales"|  4600|    2|
|        Maria|   "Finance"|  3000|    1|
|        Scott|   "Finance"|  3300|    1|
|          Jen|   "Finance"|  3900|    2|
+-------------+------------+------+-----+



In [13]:
#  PySpark Window Analytic functions

# cume_dist 

from pyspark.sql.functions import cume_dist    
df.withColumn("cume_dist",cume_dist().over(windowSpec)).show()

+-------------+------------+------+------------------+
|employee_name|  department|salary|         cume_dist|
+-------------+------------+------+------------------+
|        Kumar| "Marketing"|  2000|               0.5|
|         Jeff| "Marketing"|  3000|               1.0|
|        James|     "Sales"|  3000|               0.4|
|        James|     "Sales"|  3000|               0.4|
|       Robert|     "Sales"|  4100|               0.8|
|         Saif|     "Sales"|  4100|               0.8|
|      Michael|     "Sales"|  4600|               1.0|
|        Maria|   "Finance"|  3000|0.3333333333333333|
|        Scott|   "Finance"|  3300|0.6666666666666666|
|          Jen|   "Finance"|  3900|               1.0|
+-------------+------------+------+------------------+



In [14]:
# lag Window Function

from pyspark.sql.functions import lag    
df.withColumn("lag",lag("salary",2).over(windowSpec)).show()

+-------------+------------+------+-----+
|employee_name|  department|salary|  lag|
+-------------+------------+------+-----+
|        Kumar| "Marketing"|  2000| null|
|         Jeff| "Marketing"|  3000| null|
|        James|     "Sales"|  3000| null|
|        James|     "Sales"|  3000| null|
|       Robert|     "Sales"|  4100| 3000|
|         Saif|     "Sales"|  4100| 3000|
|      Michael|     "Sales"|  4600| 4100|
|        Maria|   "Finance"|  3000| null|
|        Scott|   "Finance"|  3300| null|
|          Jen|   "Finance"|  3900| 3000|
+-------------+------------+------+-----+



In [15]:

from pyspark.sql.functions import lead    
df.withColumn("lead",lead("salary",2).over(windowSpec)).show()

+-------------+------------+------+-----+
|employee_name|  department|salary| lead|
+-------------+------------+------+-----+
|        Kumar| "Marketing"|  2000| null|
|         Jeff| "Marketing"|  3000| null|
|        James|     "Sales"|  3000| 4100|
|        James|     "Sales"|  3000| 4100|
|       Robert|     "Sales"|  4100| 4600|
|         Saif|     "Sales"|  4100| null|
|      Michael|     "Sales"|  4600| null|
|        Maria|   "Finance"|  3000| 3900|
|        Scott|   "Finance"|  3300| null|
|          Jen|   "Finance"|  3900| null|
+-------------+------------+------+-----+



In [16]:
#  PySpark Window Aggregate Functions

windowSpecAgg  = Window.partitionBy("department")

from pyspark.sql.functions import col,avg,sum,min,max,row_number 

df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()

+------------+------+-------+-----+-----+
|  department|   avg|    sum|  min|  max|
+------------+------+-------+-----+-----+
| "Marketing"|2500.0| 5000.0| 2000| 3000|
|     "Sales"|3760.0|18800.0| 3000| 4600|
|   "Finance"|3400.0|10200.0| 3000| 3900|
+------------+------+-------+-----+-----+

